어려웠던 문제를 복습하는 시간을 가져봅시다

# Day1 - (HARD) Weather Observation Station 5 - 서브쿼리인듯?
Query the two cities in STATION with the shortest and longest CITY names, as well as their respective lengths (i.e.: number of characters in the name). If there is more than one smallest or largest city, choose the one that comes first when ordered alphabetically.

In [ ]:
# 한 놈은 가장 짧은 놈
select city, length(city) from station
where length(city) = (
    select min(CHAR_LENGTH(city)) from station 
)
order by city
limit 1;


# 한 놈은 가장 긴 놈
select city, length(city) from station
where length(city) = (
    select max(CHAR_LENGTH(city)) from station 
)
order by city
limit 1;

# 정답!

# Day1 - [~~로 시작하는 글자 찾기] Weather Observation Station 6

Query the list of CITY names starting with vowels (i.e., a, e, i, o, or u) from STATION. Your result cannot contain duplicates.

In [ ]:
select distinct(city) from station
# 정규표현식 Regular Expression? 줄여서 RegExp 
select distinct(city) from station
where REGEXP_LIKE(city,"^a|^e|^i|^o|^u")
# 정답!
# | 를 여러개 쓰지 말고
# REGEXP_LIKE(city,'^[aeiouAEIOU]') 이렇게하는게 더 깔끔하네

# Day2 - [★조건문 CASE Hard] Type of Triangle

Write a query identifying the type of each record in the TRIANGLES table using its three side lengths. Output one of the following statements for each record in the table:

Equilateral: It's a triangle with  sides of equal length.
Isosceles: It's a triangle with  sides of equal length.
Scalene: It's a triangle with  sides of differing lengths.
Not A Triangle: The given values of A, B, and C don't form a triangle.

In [ ]:
# https://velog.io/@inyong_pang/MySQL-MySQL-variables%EB%B3%80%EC%88%98-%EB%A7%8C%EB%93%A4%EC%96%B4%EC%84%9C-%EC%9D%91%EC%9A%A9%ED%95%98%EA%B8%B0getidPK
# ㄴ mysql 변수 만들어서 응용하기
# 기대값 4종류 : 정삼각형, 이등변 삼각형, 이상한 삼각형, 삼각형이 아님
SET @return = 1

select @return,
case
  when (a+b < c) or (a+c < b) or (b+c < a) then @return := "Not A Triangle"
  when (a=b & b > c) or (a=c & c > b) or (c=b & b > a) then @return := "Isosceles"
  when (a=b & b = c) then @return := "Equilateral"
  else @return := "Scalene"
end
from TRIANGLES 

# ㄴ case랑 when이랑 헷갈린거 같다 ;;
select case
  when (a+b <= c) or (a+c <= b) or (b+c <= a) then "Not A Triangle"
  when a=b & b = c then "Equilateral"
  # 헉! AND 랑 && 랑 같은거였다. &는 틀림. 논리연산자로서 &&.
  when (a=b & 2b > c) or (a=c & 2c > b) or (c=b & 2b > a) then "Isosceles"
  else "Scalene"
end
from TRIANGLES 

# 다른 사람의 정답
SELECT
  CASE
      WHEN a+b <= c OR a+c <= b OR b+c <= a THEN 'Not A Triangle'
      WHEN a=b AND b=c THEN 'Equilateral'
      WHEN a=b OR b=c OR c=a THEN 'Isosceles'
      ELSE 'Scalene'
      END
FROM triangles;


# [★MEDIAN -> @rowindex] Weather Observation Station 20

A median is defined as a number separating the higher half of a data set from the lower half. Query the median of the Northern Latitudes (LAT_N) from STATION and round your answer to decimal places.

In [ ]:
# 어떻게 중앙값을 고를까?
# ROW 개수를 얻는다
# LAT_N 기준으로 내림차순 정렬 후, 중앙 순서의 LAT_N 값을 얻는다.

# n번째 row를 얻는법
# SELECT * 
# FROM ( 
#     SELECT 
#         2@row := @row +1 AS rownum, [column name] 
#     FROM ( 
#         SELECT @row :=0) r, [table name] 
#     ) ranked 
# WHERE rownum % [n] = 1 

SET @N := 0;
SELECT COUNT(*) FROM STATION INTO @TOTAL;
# 전체 행이 99개라면 @Total = 99

SELECT
    ROUND(AVG(A.LAT_N), 4)
FROM (SELECT @N := @N +1 AS ROW_ID, LAT_N  # ROW_ID = 50인 LAT_N을 가져온다
      FROM STATION 
      ORDER BY LAT_N) A
WHERE
    # 중앙값 로직
    CASE WHEN MOD(@TOTAL, 2) = 0 
    # mod : 2로 나눈 나머지
            THEN A.ROW_ID IN (@TOTAL/2, (@TOTAL/2+1))
            ELSE A.ROW_ID = (@TOTAL+1)/2 # ROW_ID = 50
    END

set @rowindex := -1;
select round(avg(lat_n),4)
from
(select @rowindex:=@rowindex+1 as rowindex, lat_n 
from station
order by lat_n) as l 
where l.rowindex in (floor(@rowindex/2), ceil(@rowindex/2));

# [Hard GROUP BY] The PADS

Generate the following two result sets:

**Query an alphabetically ordered list of all names in OCCUPATIONS**, immediately followed by the first letter of each profession as a parenthetical (i.e.: enclosed in parentheses). For example: AnActorName(A), ADoctorName(D), AProfessorName(P), and ASingerName(S). 

**Query the number of ocurrences of each occupation in OCCUPATIONS.** Sort the occurrences in ascending order, and output them in the following format:

There are a total of [occupation_count] [occupation]s. where [occupation_count] is the number of occurrences of an occupation in OCCUPATIONS and [occupation] is the lowercase occupation name. If more than one Occupation has the same [occupation_count], they should be ordered alphabetically.

In [ ]:
# group by는 많이 고민해봐서 풀 수 있을듯?
# 너무 쉽네
# 이어붙이기 : concat
select name, CONCAT('(',UPPER(SUBSTR(OCCUPATION,1,1)),')') FROM OCCUPATIONS
from occupations
order by name;

select concat('There are a total of',' ',count(occupation),' ',lower(occupation),'s.') 
from occupations
group by occupation
order by count(occupation), occupation

# [★Too Hard Pivot] Occupations

**Pivot the Occupation column in OCCUPATIONS so that each Name is sorted alphabetically and displayed underneath its corresponding Occupation.** The output column headers should be Doctor, Professor, Singer, and Actor, respectively.

Note: Print NULL when there are no more names corresponding to an occupation.

In [ ]:
# 피봇이란게 row랑 col을 바꾸는건데,
# Doctor | Prof | Singer | Actor | 
# 이런식으로 row를 col로 바꿔야 한다.
# 그렇다면 row의 distinct 값을 col로 바꾼다.
# 그리고 새로운 row가 될 index를 세야 한다.

# 다른 사람의 답
set @r1=0, @r2=0, @r3=0, @r4=0;
select min(Doctor), min(Professor), min(Singer), min(Actor)
from(
  select case when Occupation='Doctor' then (@r1:=@r1+1)
            when Occupation='Professor' then (@r2:=@r2+1)
            when Occupation='Singer' then (@r3:=@r3+1)
            when Occupation='Actor' then (@r4:=@r4+1) end as RowNumber,
    case when Occupation='Doctor' then Name end as Doctor,
    case when Occupation='Professor' then Name end as Professor,
    case when Occupation='Singer' then Name end as Singer,
    case when Occupation='Actor' then Name end as Actor
  from OCCUPATIONS
  order by Name
) Temp
group by RowNumber
# https://www.hackerrank.com/challenges/occupations/forum
#---------------------------------------------------------------------
-- MySQL
SELECT 
MIN(CASE WHEN Occupation = 'Doctor' THEN Name ELSE NULL END) AS Doctor,
MIN(CASE WHEN Occupation = 'Professor' THEN Name ELSE NULL END) AS Professor,
MIN(CASE WHEN Occupation = 'Singer' THEN Name ELSE NULL END) AS Singer,
MIN(CASE WHEN Occupation = 'Actor' THEN Name ELSE NULL END) AS Actor
FROM (
  SELECT a.Occupation,
         a.Name,
         (SELECT COUNT(*) 
            FROM Occupations AS b
            WHERE a.Occupation = b.Occupation AND a.Name > b.Name) AS rank
  FROM Occupations AS a
) AS c
GROUP BY c.rank;
#---------------------------------------------------------------------
SELECT a.Occupation,
        a.Name,
        (
          SELECT COUNT(*) 
          FROM Occupations AS b
          WHERE a.Occupation = b.Occupation AND a.Name > b.Name
         ) AS rank
FROM Occupations AS a
+------------+-----------+------+
| Occupation | Name      | rank |
+------------+-----------+------+
| Doctor     | Samantha  |    1 |
| Actor      | Julia     |    1 |
| Actor      | Maria     |    2 |
| Singer     | Meera     |    0 |
#--------------------------------------------------------------------
SELECT 
rank,
CASE WHEN Occupation = 'Doctor' THEN Name ELSE NULL END AS Doctor,
CASE WHEN Occupation = 'Professor' THEN Name ELSE NULL END AS Professor,
CASE WHEN Occupation = 'Singer' THEN Name ELSE NULL END AS Singer,
CASE WHEN Occupation = 'Actor' THEN Name ELSE NULL END AS Actor
FROM (
  SELECT a.Occupation,
         a.Name,
         (SELECT COUNT(*) 
            FROM Occupations AS b
            WHERE a.Occupation = b.Occupation AND a.Name > b.Name) AS rank
  FROM Occupations AS a
) AS c
+------+----------+-----------+--------+-------+
| rank | Doctor   | Professor | Singer | Actor |
+------+----------+-----------+--------+-------+
|    1 | Samantha | NULL      | NULL   | NULL  |
|    1 | NULL     | NULL      | NULL   | Julia |
|    2 | NULL     | NULL      | NULL   | Maria |
|    0 | NULL     | NULL      | Meera  | NULL  |
|    0 | NULL     | Ashely    | NULL   | NULL  |
#--------------------------------------------------------------------
SELECT 
rank,
MAX(CASE WHEN Occupation = 'Doctor' THEN Name ELSE NULL END) AS Doctor,
MAX(CASE WHEN Occupation = 'Professor' THEN Name ELSE NULL END) AS Professor,
MAX(CASE WHEN Occupation = 'Singer' THEN Name ELSE NULL END) AS Singer,
MAX(CASE WHEN Occupation = 'Actor' THEN Name ELSE NULL END) AS Actor
FROM (
  SELECT a.Occupation,
         a.Name,
         (SELECT COUNT(*) 
            FROM Occupations AS b
            WHERE a.Occupation = b.Occupation AND a.Name > b.Name) AS rank
  FROM Occupations AS a
) AS c
GROUP BY c.rank;

+------+----------+-----------+--------+-------+
| rank | Doctor   | Professor | Singer | Actor |
+------+----------+-----------+--------+-------+
|    0 | Jenny    | Ashely    | Meera  | Jane  |
|    1 | Samantha | Christeen | Priya  | Julia |
|    2 | NULL     | Ketty     | NULL   | Maria |
+------+----------+-----------+--------+-------+
#-----------------------

# MS SQL 정답
# 이쪽이 생각해내기 쉬운듯

SELECT
    [Doctor], [Professor], [Singer], [Actor]
FROM
(
    SELECT ROW_NUMBER() 
    OVER (PARTITION BY OCCUPATION ORDER BY NAME) [RowNumber], * FROM OCCUPATIONS
) AS tempTable
PIVOT
(
    MAX(NAME) FOR OCCUPATION IN ([Doctor], [Professor], [Singer], [Actor])
) AS pivotTable